# Binary Text Classification

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Untuk latihan kali ini kita akan menggunakan dataset Yelp yang berisi review dari beberapa restoran di Amerika Serikat. Dataset terdiri dari 2 kelas yaitu 0 dan 1 yang menunjukkan apakah review tersebut positif atau negatif. Dataset dapat diunduh pada [tautan](https://www.kaggle.com/marklvl/sentiment-labelled-sentences-data-set?) ini.

## Load Dataset
Setelah dataset diunduh, kita load dataset pada Colaboratory

In [ ]:
    import pandas as pd
    df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dicoding/dataset/sentiment labelled sentences/yelp_labelled.txt', names=['sentence', 'label'], sep='\t')

In [ ]:
df.tail()

,sentence,label
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0
999,"Then, as if I hadn't wasted enough of my life ...",0


## Split Dataset
Setelah itu kita bagi dataset menjadi train dan test set.

In [ ]:
    from sklearn.model_selection import train_test_split
    kalimat = df['sentence'].values
    y = df['label'].values
    kalimat_latih, kalimat_test, y_latih, y_test = train_test_split(kalimat, y, test_size=0.2)

## Tokenizer
Agar teks dapat dipahami oleh model, kita harus lakukan tokenisasi. Gunakan fungsi tokenizer pada data latih dan data test. Jangan lupa gunakan fungsi pad_sequences agar setiap sequence sama panjang.

In [ ]:
    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences
     
    tokenizer = Tokenizer(num_words=250, oov_token='x')
    tokenizer.fit_on_texts(kalimat_latih) 
    tokenizer.fit_on_texts(kalimat_test)
     
    sekuens_latih = tokenizer.texts_to_sequences(kalimat_latih)
    sekuens_test = tokenizer.texts_to_sequences(kalimat_test)
     
    padded_latih = pad_sequences(sekuens_latih) 
    padded_test = pad_sequences(sekuens_test)

## Embedding
Untuk arsitektur yang akan digunakan adalah layer embedding, dengan argumen pertama sesuai dengan jumlah vocabulary/kata yang kita pakai pada tokenizer. Argumen selanjutnya adalah dimensi embedding, dan input_length yang merupakan panjang dari sequence. Nah di kita tidak menggunakan layer Flatten melainkan kita menggantinya dengan GlobalAveragePooling1D. Fungsi ini bekerja lebih baik pada kasus NLP dibanding Flatten.

In [ ]:
    import tensorflow as tf
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(250, 16, input_length=20),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

## Training Model

Setelah arsitektur model dibentuk, dan loss function serta optimizer ditentukan, kita dapat memulai pelatihan model kita. Di sini penulis menggunakan 30 epoch. Anda bebas bereksperimen dengan nilai yang lain.

In [ ]:
    num_epochs = 30
    history = model.fit(padded_latih, y_latih, epochs=num_epochs, 
                        validation_data=(padded_test, y_test), verbose=2)

Epoch 1/30
25/25 - 3s - loss: 0.6930 - accuracy: 0.5213 - val_loss: 0.6922 - val_accuracy: 0.5700
Epoch 2/30
25/25 - 0s - loss: 0.6918 - accuracy: 0.5987 - val_loss: 0.6911 - val_accuracy: 0.6250
Epoch 3/30
25/25 - 0s - loss: 0.6902 - accuracy: 0.5975 - val_loss: 0.6891 - val_accuracy: 0.6350
Epoch 4/30
25/25 - 0s - loss: 0.6869 - accuracy: 0.6012 - val_loss: 0.6849 - val_accuracy: 0.5950
Epoch 5/30
25/25 - 0s - loss: 0.6816 - accuracy: 0.6275 - val_loss: 0.6802 - val_accuracy: 0.6350
Epoch 6/30
25/25 - 0s - loss: 0.6751 - accuracy: 0.6750 - val_loss: 0.6729 - val_accuracy: 0.5950
Epoch 7/30
25/25 - 0s - loss: 0.6654 - accuracy: 0.6587 - val_loss: 0.6638 - val_accuracy: 0.6300
Epoch 8/30
25/25 - 0s - loss: 0.6509 - accuracy: 0.7237 - val_loss: 0.6506 - val_accuracy: 0.6400
Epoch 9/30
25/25 - 0s - loss: 0.6310 - accuracy: 0.7075 - val_loss: 0.6362 - val_accuracy: 0.6900
Epoch 10/30
25/25 - 0s - loss: 0.6110 - accuracy: 0.7450 - val_loss: 0.6158 - val_accuracy: 0.6600
Epoch 11/30
25/25 -

Hasil dari pelatihan model kita menunjukkan akurasi yang cukup baik dengan data yang sangat sedikit.